In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from epiweeks import Week
from datetime import date, datetime
from os.path import dirname, join, isfile, isdir

from bulletin.utils import static
from bulletin.utils.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf, normalize_igbe, normalize_labels
from bulletin.utils.utils import isvaliddate

from bulletin.utils.utils import Timer, auto_fit_columns
output = join("output")

In [8]:
internamento = pd.read_excel(join('input','internamento.xls'))
internamento.columns = [ normalize_labels(x) for x in internamento.columns]
internamento['leito_tipo'] = internamento['leito_tipo'].apply(normalize_text)
internamento.loc[internamento['leito_tipo']=='CORONAVIRUS-UTI ADULTO (MS)', 'leito_tipo'] = 'CORONAVIRUS-UTI ADULTO'
internamento['motivo_alta'] = internamento['motivo_alta'].apply(normalize_text)
# internamento['ano'] = internamento.apply(lambda row: Week.fromdate(row['data_alta']).year, axis=1)
# internamento['mes'] = internamento.apply(lambda row: Week.fromdate(row['data_alta']).startdate().month, axis=1)
# internamento['se'] = internamento.apply(lambda row: Week.fromdate(row['data_alta']).week, axis=1)
internamento[['tempo_internamento_(em_dias)','idade']].mean()

tempo_internamento_(em_dias)     8.324503
idade                           59.984208
dtype: float64

In [3]:
media_internamento = internamento.groupby('leito_tipo')[['idade','tempo_internamento_(em_dias)']].mean()
aux_df = pd.DataFrame(columns=media_internamento.columns, index=['TOTAL'])
aux_df.loc['TOTAL'] = alta_internamento.sum()
alta_internamento = alta_internamento.append(aux_df)
alta_internamento['percent'] = alta_internamento.apply(lambda row: row['qtde'] / alta_internamento.loc['TOTAL','qtde'], axis=1)
alta_internamento
media_internamento

,idade,tempo_internamento_(em_dias)
leito_tipo,,
CORONAVIRUS-CLINICA GERAL,58.218421,7.586842
CORONAVIRUS-UTI ADULTO,66.042889,10.855530


In [4]:
alta_internamento = internamento.groupby('motivo_alta')[['idade']].count().rename(columns={'idade':'qtde'})
aux_df = pd.DataFrame(columns=['qtde'], index=['TOTAL'])
aux_df.loc['TOTAL'] = alta_internamento.sum()
alta_internamento = alta_internamento.append(aux_df)
alta_internamento['percent'] = alta_internamento.apply(lambda row: row['qtde'] / alta_internamento.loc['TOTAL','qtde'], axis=1)
alta_internamento

,qtde,percent
ALTA ADMINISTRATIVA,10,0.005094
ALTA MEDICA,1468,0.747835
ALTA POR OBITO,379,0.193072
ALTA POR PEDIDO DO PACIENTE OU FAMILIAR,4,0.002038
ALTA POR TRANSFERENCIA PARA OUTRO HOSPITAL,97,0.049414
EVASAO DO PACIENTE,5,0.002547
TOTAL,1963,1.000000


In [5]:
writer = pd.ExcelWriter(join(output,"internamentos.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

media_internamento.to_excel(writer,sheet_name='media_internamento')
alta_internamento.to_excel(writer,sheet_name='alta_internamento')

worksheet = writer.sheets[f"media_internamento"]
auto_fit_columns(worksheet,media_internamento)

worksheet = writer.sheets[f"alta_internamento"]
auto_fit_columns(worksheet,alta_internamento)

writer.save()
writer.close()